In [2]:
import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the reference image and the image to be aligned
full_image = cv2.imread('ref_image.jpg')
aligned_face = cv2.imread('anakin.png')

# Convert the full image to grayscale
gray = cv2.cvtColor(full_image, cv2.COLOR_BGR2GRAY)

# Detect the face using dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./temp/shape_predictor_68_face_landmarks.dat')
rects = detector(gray, 1)

# Assume there is only one face in the image
face_rect = rects[0]

# Get the facial landmarks of the face in the full image
landmarks = predictor(gray, face_rect)

# Get the alignment transformation from the landmarks
src_points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])
dst_points = np.array([(int(p), int(q)) for p, q in zip(src_points[:, 0] / 2, src_points[:, 1] / 2)])
M = cv2.estimateAffinePartial2D(dst_points, src_points)[0]

# Invert the alignment transformation to get the de-alignment transformation
M_inv = cv2.invertAffineTransform(M)

# Apply the de-alignment transformation to the aligned face image
dealigned_face = cv2.warpAffine(aligned_face, M_inv, (full_image.shape[1], full_image.shape[0]))

# Replace the face region in the full image with the de-aligned face
x, y, w, h = face_rect.left(), face_rect.top(), face_rect.width(), face_rect.height()
full_image[y:y+h, x:x+w] = dealigned_face

# Display the original image, the aligned image, and the back-projected image
# Save the images
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(full_image, cv2.COLOR_BGR2RGB))
plt.title('Back-Projected Image')
plt.axis('off')
plt.tight_layout()
plt.savefig('back_proj_image.png', dpi=300)

ModuleNotFoundError: No module named 'cv2'